<a href="https://colab.research.google.com/github/manabil/Machine_Learning_Intermediate/blob/main/3-%20Natural%20Language%20Processing/3F-%20Multiclass%20Text%20Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

In [6]:
df = pd.read_csv('imdb_indonesian_movies_2.csv')
df = df.drop(columns=['judul_film'])

In [7]:
category = pd.get_dummies(df.genre)
df_baru = pd.concat([df, category], axis=1)
df_baru = df_baru.drop(columns='genre')
df_baru

,ringkasan_sinopsis,Drama,Horor,Komedi,Laga,Romantis
0,Raden Mas Said putra sulung Tumenggung Wilarik...,1,0,0,0,0
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,1,0,0,0,0
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,1,0,0,0,0
3,POL menceritakan kisah hidup yang luar biasa d...,1,0,0,0,0
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,1,0,0,0,0
...,...,...,...,...,...,...
1000,Winter in Tokyo berpusat pada kehidupan Ishida...,0,0,0,0,1
1001,Markonah melarikan diri ke Jakarta karena akan...,0,0,0,0,1
1002,"Tempat aking lebih dari 36 jam, Last Night ada...",0,0,0,0,1
1003,Proyek baru ini adalah tentang seorang lelaki ...,0,0,0,0,1


In [8]:
sinopsis = df_baru['ringkasan_sinopsis'].values
label = df_baru[['Drama', 'Horor', 'Komedi', 'Laga', 'Romantis']].values

In [30]:
longest_sentences = []
for i in sinopsis:
    longest_sentences.append(len(i.split(' ')))
max(longest_sentences)

1729

In [15]:
sinopsis_latih, sinopsis_test, label_latih, label_test = train_test_split(sinopsis, label, test_size=0.2)

In [32]:
print(len(sinopsis_latih))
print(len(label_latih))
print(len(sinopsis_test))
print(len(label_test))

804
804
201
201


In [66]:
vocab_size = 15000
embedding_dim = 16
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 20000

In [74]:
# Create Tokenizer
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(sinopsis_latih) 
tokenizer.fit_on_texts(sinopsis_test)
    
# Create Sequnces
sekuens_latih = tokenizer.texts_to_sequences(sinopsis_latih)
sekuens_test = tokenizer.texts_to_sequences(sinopsis_test)

# Choose Max Sequences
max_sequence_latih = max([len(x) for x in sekuens_latih])
max_sequence_test = max([len(x) for x in sekuens_test])
max_length = max_sequence_latih if max_sequence_latih > max_sequence_test else max_sequence_test
print(f'Max Sequences\t= {max_length} word')
    
# Padding Sequences
padded_latih = pad_sequences(sekuens_latih, maxlen=max_length, padding=padding_type, truncating=trunc_type) 
padded_test = pad_sequences(sekuens_test, maxlen=max_length, padding=padding_type, truncating=trunc_type)

total_word = len(tokenizer.word_index)
print(f'Word Total\t= {total_word} word')

Max Sequences	= 1765 word
Word Total	= 13678 word


In [75]:
print(padded_latih[0].shape)
print(padded_test[0].shape)

(1765,)
(1765,)


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=16),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
num_epochs = 30
history = model.fit(padded_latih, label_latih, epochs=num_epochs, 
                    validation_data=(padded_test, label_test), verbose=2)

Epoch 1/30
26/26 - 19s - loss: 1.6108 - accuracy: 0.1978 - val_loss: 1.6126 - val_accuracy: 0.1841 - 19s/epoch - 741ms/step
Epoch 2/30
26/26 - 17s - loss: 1.6066 - accuracy: 0.2562 - val_loss: 1.6112 - val_accuracy: 0.1990 - 17s/epoch - 664ms/step
Epoch 3/30
26/26 - 16s - loss: 1.5569 - accuracy: 0.3595 - val_loss: 1.6535 - val_accuracy: 0.2438 - 16s/epoch - 626ms/step
Epoch 4/30
26/26 - 17s - loss: 1.3796 - accuracy: 0.4067 - val_loss: 1.6308 - val_accuracy: 0.2488 - 17s/epoch - 644ms/step
Epoch 5/30
26/26 - 17s - loss: 1.0280 - accuracy: 0.5871 - val_loss: 2.0030 - val_accuracy: 0.2338 - 17s/epoch - 636ms/step
Epoch 6/30
26/26 - 16s - loss: 0.6599 - accuracy: 0.7438 - val_loss: 2.5900 - val_accuracy: 0.2687 - 16s/epoch - 625ms/step
Epoch 7/30
26/26 - 17s - loss: 0.4561 - accuracy: 0.8333 - val_loss: 3.1703 - val_accuracy: 0.2836 - 17s/epoch - 638ms/step
Epoch 8/30
26/26 - 17s - loss: 0.2431 - accuracy: 0.9353 - val_loss: 3.6170 - val_accuracy: 0.3333 - 17s/epoch - 640ms/step
Epoch 9/

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=16),
    tf.keras.layers.LSTM(64, dropout=0.3, recurrent_dropout=0.3),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
num_epochs = 30
history = model.fit(padded_latih, label_latih, epochs=num_epochs, 
                    validation_data=(padded_test, label_test), verbose=2)

Epoch 1/30
26/26 - 28s - loss: 1.6101 - accuracy: 0.2090 - val_loss: 1.6120 - val_accuracy: 0.1741 - 28s/epoch - 1s/step
Epoch 2/30
26/26 - 23s - loss: 1.6077 - accuracy: 0.2214 - val_loss: 1.6109 - val_accuracy: 0.2040 - 23s/epoch - 900ms/step
Epoch 3/30
26/26 - 25s - loss: 1.5867 - accuracy: 0.3109 - val_loss: 1.5954 - val_accuracy: 0.2139 - 25s/epoch - 966ms/step
Epoch 4/30
26/26 - 24s - loss: 1.4427 - accuracy: 0.3557 - val_loss: 1.6059 - val_accuracy: 0.2537 - 24s/epoch - 905ms/step
Epoch 5/30
26/26 - 23s - loss: 1.1838 - accuracy: 0.4117 - val_loss: 1.7846 - val_accuracy: 0.2438 - 23s/epoch - 891ms/step
Epoch 6/30
26/26 - 23s - loss: 1.0414 - accuracy: 0.5112 - val_loss: 1.8504 - val_accuracy: 0.2637 - 23s/epoch - 897ms/step
Epoch 7/30
26/26 - 24s - loss: 0.8688 - accuracy: 0.5647 - val_loss: 1.9150 - val_accuracy: 0.2388 - 24s/epoch - 908ms/step
Epoch 8/30
26/26 - 24s - loss: 0.6982 - accuracy: 0.7264 - val_loss: 2.1951 - val_accuracy: 0.2637 - 24s/epoch - 938ms/step
Epoch 9/30


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=16),
    tf.keras.layers.LSTM(64, dropout=0.5, recurrent_dropout=0.5),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
num_epochs = 30
history = model.fit(padded_latih, label_latih, epochs=num_epochs, 
                    validation_data=(padded_test, label_test), verbose=2)

Epoch 1/30
26/26 - 27s - loss: 1.6099 - accuracy: 0.1766 - val_loss: 1.6108 - val_accuracy: 0.1791 - 27s/epoch - 1s/step
Epoch 2/30
26/26 - 24s - loss: 1.6085 - accuracy: 0.2127 - val_loss: 1.6121 - val_accuracy: 0.1741 - 24s/epoch - 905ms/step
Epoch 3/30
26/26 - 23s - loss: 1.6047 - accuracy: 0.2301 - val_loss: 1.6089 - val_accuracy: 0.2090 - 23s/epoch - 899ms/step
Epoch 4/30
26/26 - 24s - loss: 1.5691 - accuracy: 0.3346 - val_loss: 1.6087 - val_accuracy: 0.2687 - 24s/epoch - 932ms/step
Epoch 5/30
26/26 - 25s - loss: 1.3281 - accuracy: 0.4627 - val_loss: 1.6946 - val_accuracy: 0.2886 - 25s/epoch - 949ms/step
Epoch 6/30
26/26 - 24s - loss: 1.1801 - accuracy: 0.5112 - val_loss: 1.6878 - val_accuracy: 0.2488 - 24s/epoch - 909ms/step
Epoch 7/30
26/26 - 24s - loss: 0.9199 - accuracy: 0.6493 - val_loss: 1.8903 - val_accuracy: 0.2985 - 24s/epoch - 922ms/step
Epoch 8/30
26/26 - 23s - loss: 0.7537 - accuracy: 0.7027 - val_loss: 2.0204 - val_accuracy: 0.2587 - 23s/epoch - 886ms/step
Epoch 9/30


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=16),
    tf.keras.layers.LSTM(64, dropout=0.9, recurrent_dropout=0.9),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
num_epochs = 30
history = model.fit(padded_latih, label_latih, epochs=num_epochs, 
                    validation_data=(padded_test, label_test), verbose=2)

Epoch 1/30
26/26 - 26s - loss: 1.6104 - accuracy: 0.1766 - val_loss: 1.6107 - val_accuracy: 0.1741 - 26s/epoch - 1s/step
Epoch 2/30
26/26 - 23s - loss: 1.6094 - accuracy: 0.2040 - val_loss: 1.6106 - val_accuracy: 0.1741 - 23s/epoch - 899ms/step
Epoch 3/30
26/26 - 24s - loss: 1.6089 - accuracy: 0.2077 - val_loss: 1.6108 - val_accuracy: 0.1940 - 24s/epoch - 906ms/step
Epoch 4/30
26/26 - 25s - loss: 1.6076 - accuracy: 0.2114 - val_loss: 1.6124 - val_accuracy: 0.2189 - 25s/epoch - 962ms/step
Epoch 5/30
26/26 - 23s - loss: 1.6082 - accuracy: 0.2077 - val_loss: 1.6121 - val_accuracy: 0.1841 - 23s/epoch - 894ms/step
Epoch 6/30
26/26 - 24s - loss: 1.6063 - accuracy: 0.2214 - val_loss: 1.6121 - val_accuracy: 0.2040 - 24s/epoch - 919ms/step
Epoch 7/30
26/26 - 24s - loss: 1.6011 - accuracy: 0.2500 - val_loss: 1.6143 - val_accuracy: 0.2338 - 24s/epoch - 910ms/step
Epoch 8/30
26/26 - 23s - loss: 1.5912 - accuracy: 0.2537 - val_loss: 1.6095 - val_accuracy: 0.2289 - 23s/epoch - 889ms/step
Epoch 9/30


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=16),
    tf.keras.layers.LSTM(64, dropout=0.7, recurrent_dropout=0.7),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
num_epochs = 30
history = model.fit(padded_latih, label_latih, epochs=num_epochs, 
                    validation_data=(padded_test, label_test), verbose=2)

Epoch 1/30
26/26 - 70s - loss: 1.6102 - accuracy: 0.1915 - val_loss: 1.6102 - val_accuracy: 0.1592 - 70s/epoch - 3s/step
Epoch 2/30
26/26 - 67s - loss: 1.6085 - accuracy: 0.2326 - val_loss: 1.6095 - val_accuracy: 0.1891 - 67s/epoch - 3s/step
Epoch 3/30
26/26 - 67s - loss: 1.6042 - accuracy: 0.2512 - val_loss: 1.6124 - val_accuracy: 0.1692 - 67s/epoch - 3s/step
Epoch 4/30
26/26 - 66s - loss: 1.5734 - accuracy: 0.2861 - val_loss: 1.6151 - val_accuracy: 0.2040 - 66s/epoch - 3s/step
Epoch 5/30
26/26 - 68s - loss: 1.4666 - accuracy: 0.3433 - val_loss: 1.5925 - val_accuracy: 0.2139 - 68s/epoch - 3s/step
Epoch 6/30
26/26 - 66s - loss: 1.3536 - accuracy: 0.3818 - val_loss: 1.5813 - val_accuracy: 0.2189 - 66s/epoch - 3s/step
Epoch 7/30
26/26 - 69s - loss: 1.2570 - accuracy: 0.4229 - val_loss: 1.6250 - val_accuracy: 0.3035 - 69s/epoch - 3s/step
Epoch 8/30
26/26 - 66s - loss: 1.1267 - accuracy: 0.5149 - val_loss: 1.7133 - val_accuracy: 0.2239 - 66s/epoch - 3s/step
Epoch 9/30
26/26 - 67s - loss: 1

In [ ]:
cobo = ['Dalam perjalanan pulang naik kereta di Jerman Arini (Aura Kasih) mahasiswa 38 tahun bertemu dengan Nick (Morgan Oey) 15 tahun lebih muda yang melakukan perjalanan backpacking keliling Eropa. Nick jatuh cinta pada pandangan pertama dan segera saja berusaha masuk ke dalam hidup Arini sementara Arini yang tak lagi percaya akan cinta tak mampu menerima Nick. Beberapa bulan kemudian Arini kembali ke Indonesia dan diangkat menjadi CEO di perusahaan tempatnya bekerja. Nick yang terdesak oleh perasaan cintanya pada Arini menyusulnya pulang ke Yogyakarta meninggalkan kuliahnya di London. Di kota inilah mereka memulai kembali hubungan mereka berhadapan-hadapan dengan masa lalu gelap Arini bersama mantan suaminya Helmi (Haydar Salishz) dan sahabatnya Ira (Olga Lydia). Seumur hidupnya Arini menyesali pertemuannya dengan Ira di kereta menuju Yogyakarta beberapa tahun silam sementara Nick terus berusaha membuat Arini mengingat pertemuan mereka di kereta di Jerman']
len(cobo[0])

963

In [ ]:
hasil = model.predict(cobo)
hasil

ValueError: ignored